# 袁倫祥醫師 GPT-SoVITS 訓練
1. Runtime > Change runtime type > GPU (T4)
2. 依序執行每個 cell

In [ ]:
!git clone https://github.com/RVC-Boss/GPT-SoVITS.git
%cd GPT-SoVITS
!bash install.sh --device CU126 --source HF

In [ ]:
# 上傳 yuan_training_data.zip
from google.colab import files
uploaded = files.upload()
!unzip -o yuan_training_data.zip -d training_data/

In [ ]:
# 啟動 WebUI 進行訓練
!python webui.py